In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from genotypes import PRIMITIVES
from genotypes import Genotype
from operations import *
import numpy as np

In [2]:
tf.enable_eager_execution()

In [3]:
class MixedOp(Model):
    """Makes mixed operations object
    """

    def __init__(self, C, stride):
        """Makes ops array of all the arrays

        Args:
            C (int): the current state
        """
        super(MixedOp, self).__init__()
        self._ops = []
        for primitive in PRIMITIVES:
                op = OPS[primitive](C, stride)
                if 'pool' in primitive:
                    if('avg' in primitive):
                        op = AvgPool3x3(C, stride)
                    elif('max' in primitive):
                        op = MaxPool3x3(C, stride)
                self._ops.append(op)
        
    def call(self, x, weights):
        """Converts the discrete set of operation into conitnuous mixed operation

        Args:
            x (tensor): can be tensor or array, (e.g. image-array)
            weights (tensor): tensor or array of Softmax probability of alphas

        Returns:
            tensor: sum of product(weights, operation(x))
        """
        return sum(w * op(x) for w, op in zip(weights, self._ops))

In [4]:
class Cell(Model):

    def __init__(self, steps, multiplier, C_prev_prev, C_prev, C, reduction, reduction_prev, upsample_prev):
        super(Cell, self).__init__()
        self.reduction = reduction

        if reduction_prev:
            self.preprocess0 = FactorizedReduce(C_prev_prev, C)
        elif upsample_prev:
            self.preprocess0 = FactorizedUp(C_prev_prev, C)
        else:
            self.preprocess0 = ReLUConvBN(C_prev_prev, C, 1, 1, 'same')
        self.preprocess1 = ReLUConvBN(C_prev, C, 1, 1, 'same')
        self._steps = steps
        self._multiplier = multiplier

        self._ops = []
        self._bns = []
        for i in range(self._steps):
            for j in range(2+i):
                stride = 2 if reduction and j < 2 else 1
                op = MixedOp(C, stride)
                self._ops.append(op)

    def call(self, s0, s1, weights):
    
        s0 = self.preprocess0(s0)
        s1 = self.preprocess1(s1)

        states = [s0, s1]
        offset = 0
    
        for i in range(self._steps):
            s = sum(self._ops[offset+j](h, weights[offset+j]) for j, h in enumerate(states))
            offset += len(states)
            states.append(s)

        return tf.concat(states[-self._multiplier:], axis=-1)


In [5]:
class UpsampleCell(Model):

    def __init__(self, steps, multiplier, C_prev_prev, C_prev, C):
        super(UpsampleCell, self).__init__()
        self.preprocess0 = ReLUConvBN(C_prev_prev, C, 1, 1, 'same')
        self.preprocess1 = ReLUConvBN(C_prev, C, 1, 1, 'same')
        self._steps = steps
        self._multiplier = multiplier
        self.UpConv = layers.Conv2DTranspose(C*self._multiplier, 
                                        kernel_size=3,
                                        strides=2,
                                        padding='same')
        self.reduction = False

    def call(self, s0, s1, weights):
        s0 = self.preprocess0(s0)
        s1 = self.preprocess1(s1)

        s0 = self.UpConv(s0)
        s1 = self.UpConv(s1)

        return s0 + s1


In [6]:
class Network(Model):

    def __init__(self, C, net_layers, criterion, steps=4, multiplier=4, stem_multiplier=3, num_classes=1):
        super(Network, self).__init__()
        self._C = C
        self._criterion = criterion
        self._steps = steps
        self._multiplier = multiplier
        self.net_layers = net_layers
        self.num_classes = num_classes
        
        C_curr = C*stem_multiplier

        # stem operation
        self.stem_op = tf.keras.Sequential()
        self.stem_op.add(tf.keras.layers.Conv2D(
            C_curr, kernel_size=3, padding='same', use_bias=False))
        self.stem_op.add(tf.keras.layers.BatchNormalization())

        C_prev_prev, C_prev, C_curr = C_curr, C_curr, C
        self.cells = []
        self.skip_ops = []

        reduction_prev = False

        # For reduction
        for i in range(self.net_layers):
            if i % 2 == 1:
                C_curr *= 2
                reduction = True
            else:
                reduction = False
                self.skip_ops += [SkipConnection(C_curr)]
            cell = Cell(steps, multiplier, C_prev_prev, C_prev, C_curr,
                        reduction,
                        reduction_prev,
                        upsample_prev=False)
            reduction_prev = reduction
            self.cells += [cell]

            C_prev_prev, C_prev = C_prev, multiplier*C_curr

        for i in range(self.net_layers-1):
            if i % 2 == 0:
                C_curr = C_curr // 2

                cell = UpsampleCell(steps, multiplier,
                                    C_prev_prev, C_prev, C_curr)
            else:
                cell = Cell(steps, multiplier, C_prev_prev, C_prev, C_curr,
                            reduction=False,
                            reduction_prev=False,
                            upsample_prev=True)
            self.cells += [cell]
            C_prev_prev, C_prev = C_prev, multiplier*C_curr
        
        self.softmaxConv = tf.keras.Sequential(name="softmaxConv")
        self.softmaxConv.add(tf.keras.layers.Conv2D(
            self.num_classes, kernel_size=1, strides=1, padding='same'))
        self.softmaxConv.add(Softmax())

        self._initialize_alphas()

    def _initialize_alphas(self):
        k = sum(1 for i in range(self._steps) for n in range(2+i))
        num_ops = len(PRIMITIVES)
        alphas_normal = lambda: 1e-3*tf.random.uniform([k, num_ops])
        alphas_reduce = lambda: 1e-3*tf.random.uniform([k, num_ops])
        self.alphas_normal = tf.Variable(
            alphas_normal, name='alphas_normal')
        self.alphas_reduce = tf.Variable(
            alphas_reduce, name='alphas_reduce')
        self._arch_parameters = [
            self.alphas_normal,
            self.alphas_reduce,
        ]

    def arch_parameters(self):
        return self._arch_parameters

    def new(self):
        model_new = Network(self._C, self.net_layers, self._criterion, num_classes=self.num_classes)
        for x, y in zip(model_new.arch_parameters(), self.arch_parameters()):
            x.assign(y)
        return model_new

    def _loss(self, logits, target):
        b, w, h, c = target.shape
        y = tf.reshape(tf.cast(target, tf.int64), (b, w, h))
        y = tf.one_hot(y, self.num_classes, on_value=1.0, off_value=0.0)
        return self._criterion(y, logits)

    def call(self, inp):
        s0 = s1 = self.stem_op(inp)
        self.arr = []
        ids = []
        pos = -1

        middle = self.net_layers - 1
        for i, cell in enumerate(self.cells):
            if cell.reduction:
                weights = tf.nn.softmax(self.alphas_reduce, axis=-1)
            else:
                weights = tf.nn.softmax(self.alphas_normal, axis=-1)

            s0, s1 = s1, cell(s0, s1, weights)

            if (i < middle and i % 2 == 0):
                self.arr.append(s1)
                ids.append(i)

            if (i > middle and i % 2 == 1):
                C_curr = s1.shape[1]
                s1 = self.skip_ops[-pos-1](self.arr[pos], s1)
                pos -= 1
        return self.softmaxConv(s1)
    
    def genotype(self, alphas_normal, alphas_reduce):
            def _parse(weights):
                gene = []
                n = 2
                start = 0
                for i in range(self._steps):
                    end = start + n
                    W = weights[start:end].copy()
                    edges = sorted(range(i + 2), key=lambda x: -max(W[x][k] for k in range(len(W[x])) if k != PRIMITIVES.index('none')))[:2]
                    for j in edges:
                        k_best = None
                        for k in range(len(W[j])):
                            if k != PRIMITIVES.index('none'):
                                if k_best is None or W[j][k] > W[j][k_best]:
                                    k_best = k
                        gene.append((PRIMITIVES[k_best], j))
                    start = end
                    n += 1
                return gene

            gene_normal = _parse(alphas_normal)
            gene_reduce = _parse(alphas_reduce)

            concat = range(2+self._steps-self._multiplier, self._steps+2)
            genotype = Genotype(
              normal=gene_normal, normal_concat=concat,
              reduce=gene_reduce, reduce_concat=concat
            )
            return genotype


## Network Test

In [7]:
criterion = tf.losses.sigmoid_cross_entropy
network = Network(3, 3, criterion)

In [8]:
np_ds_train = (np.random.randint(0, 256, (20, 16, 16, 3)).astype(np.float32), np.random.randint(0, 2, (20, 16, 16, 1)).astype(np.float32))
np_ds_valid = (np.random.randint(0, 256, (20, 16, 16, 3)).astype(np.float32), np.random.randint(0, 2, (20, 16, 16, 1)).astype(np.float32))
ds_train = tf.data.Dataset.from_tensor_slices(np_ds_train).batch(2)
ds_valid = tf.data.Dataset.from_tensor_slices(np_ds_valid).batch(2)

In [9]:
it = ds_train.make_one_shot_iterator()
image, label = it.get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [10]:
print(network(image))

InvalidArgumentError: Incompatible shapes: [2,16,16,3] vs. [2,14,14,3] [Op:AddV2] name: network/cell/mixed_op/add/

In [ ]:
ws2 = network.trainable_weights
for w in ws2:
    if 'aphas' in w.name:
        print(w)

In [ ]:
network.summary()

In [ ]:
network.get_weights()

# Testing Here!

In [7]:
import numpy as np

## Mixed Op

In [8]:
mix_op = MixedOp(3, stride=1)

In [9]:
weights = np.random.rand(8).astype(np.float32)
features = np.random.rand(2, 16, 16, 3).astype(np.float32)

init = tf.global_variables_initializer()

out = mix_op(features, weights)

In [10]:
out

<tf.Tensor: id=84, shape=(2, 16, 16, 3), dtype=float32, numpy=
array([[[[0.32285148, 0.54710644, 1.1514405 ],
         [0.42422813, 0.96005946, 1.0759336 ],
         [0.14072308, 0.90018857, 1.2677069 ],
         ...,
         [0.933166  , 0.4263308 , 0.95550275],
         [0.8583072 , 0.66756475, 1.0592568 ],
         [0.72377366, 0.19719526, 1.0426829 ]],

        [[0.8192141 , 1.1413007 , 0.9077792 ],
         [0.6702423 , 1.250958  , 1.1351435 ],
         [0.5356897 , 1.5157348 , 1.1250756 ],
         ...,
         [0.9008583 , 0.94851655, 1.0374324 ],
         [0.8049585 , 1.091066  , 1.0919341 ],
         [0.13932484, 0.6754265 , 1.5323551 ]],

        [[0.78010374, 1.1057564 , 1.0542378 ],
         [0.5173835 , 1.2058035 , 1.2370738 ],
         [0.5772717 , 1.1649121 , 1.0180331 ],
         ...,
         [0.78804624, 1.1538341 , 1.1642927 ],
         [0.686646  , 1.0289286 , 1.2927196 ],
         [0.15988475, 0.686424  , 1.5292027 ]],

        ...,

        [[1.1126094 , 0.75892

## Cell

In [11]:
cell = Cell(steps=4,
            multiplier=4,
            C_prev_prev=3,
            C_prev=3,
            C=3,
            reduction=False,
            reduction_prev=False,
            upsample_prev=False)

In [12]:
weights = np.random.rand(20,8).astype(np.float32)
s0 = np.random.rand(2, 16, 16, 3).astype(np.float32)
s1 = np.random.rand(2, 16, 16, 3).astype(np.float32)

init = tf.global_variables_initializer()

cell_out = cell(s0, s1, weights)

In [ ]:
cell_out

## Upsample Cell

In [13]:
up_cell = UpsampleCell(4, 4, 3, 3, 3)

In [15]:
s0 = np.random.rand(2, 16, 16, 3).astype(np.float32)
s1 = np.random.rand(2, 16, 16, 3).astype(np.float32)

init = tf.global_variables_initializer()

up_cell_out = up_cell(s0, s1, weights)

## Extra Test

In [ ]:
s0 = np.random.rand(2, 16, 16, 6).astype(np.float32)
conv = tf.keras.Sequential()
conv.add(ReLUConvBN(6, 6, 1, 1, 'same'))
conv.add(FactorizedReduce(6,6))
conv(s0)

In [ ]:
var = tf.Variable([[0.9], [0.5]])
var2 = tf.Variable([[0.9], [0.5]])

In [ ]:
Zero(2).name